In [1]:
import pandas as pd
import time,datetime
import numpy as np

In [26]:
#timeseries data
all_data=pd.read_csv('./data/Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

dic_data = {k: v for k, v in all_data.groupby('date')}

In [27]:
#risk:cov_mat
def risk(date_str):
    risk_data=pd.read_csv('./data/RM_Output_1/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    return risk_data,risk_sedol,risk_mat
risk_data,risk_sedol,risk_mat=risk(date_list[0])

In [29]:
set(risk_sedol)-set(a)

{'$$CASH0'}

In [28]:
a=list(dic_data[list(dic_data.keys())[0]]['sedol'])

In [3]:
all_data=pd.read_csv('./data/Timeseries_data_SP500.csv')

In [7]:
all_data['dd']=all_data['DATE']+str('kk')

In [21]:
dic_data[list(dic_data.keys())[0]]

,date,sedol,sector,beta,as,name,bw,mq
0,2007-01-03,2000019,Consumer Discretionary,1.000000,-1.350000e-05,"Amazon.com, Inc.",0.000938,2
1,2007-01-03,2002305,Health Care,1.000000,-3.390000e-06,Abbott Laboratories,0.005894,1
2,2007-01-03,2002479,Utilities,1.000000,1.480000e-05,AES Corporation,0.001143,2
3,2007-01-03,2005973,Information Technology,0.921277,1.480000e-05,International Business Machines Corporation,0.011490,1
4,2007-01-03,2007849,Information Technology,1.000000,1.520000e-05,"Advanced Micro Devices, Inc.",0.000840,3
5,2007-01-03,2008154,Information Technology,1.000000,-1.350000e-05,Adobe Systems Incorporated,0.001826,1
6,2007-01-03,2011602,Materials,1.000000,-1.230000e-05,"Air Products and Chemicals, Inc.",0.001242,2
7,2007-01-03,2017213,Health Care,1.000000,-1.880000e-05,"Allergan, Inc.",0.001436,2
8,2007-01-03,2018175,Materials,1.000000,-5.410000e-06,E. I. du Pont de Nemours and Company,0.003537,1
9,2007-01-03,2019479,Financials,1.000000,9.140000e-06,"Boston Properties, Inc.",0.001027,2
